In [1]:
import pandas as pd
import numpy as np
dataset=pd.read_csv("ckd.csv")


In [2]:
dataset=pd.get_dummies(dataset, drop_first=True)
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [3]:
independent=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]



In [4]:

dependent=dataset['classification_yes']


In [5]:

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(independent, dependent, test_size = 1/3, random_state = 0)

In [7]:
from sklearn.preprocessing import StandardScaler
SC=StandardScaler()
x_train=SC.fit_transform(x_train)
x_test=SC.transform(x_test)

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
param_grid= {'criterion':['gini', 'entropy', 'log_loss'], 'splitter':['best', 'random'], 'min_samples_split':[2,5,1.,20,50,100],'min_samples_leaf':[0.001,0.01,0.1,1.0,10.0,100.0,1000.0],'max_features':['auto','sqrt','log2'],'class_weight':['dict','balanced']}

grid=GridSearchCV(DecisionTreeClassifier(),param_grid,scoring='f1_weighted')
grid.fit(x_train,y_train)

C:\Users\poorn\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
6480 fits failed out of a total of 7560.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3780 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\poorn\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\poorn\Anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 942, in fit
    X_idx_sorted=X_idx_sorted,
  File "C:\Users\poorn\Anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 221, in fit
    self.class_weight, y_original
  File "C:\Users\poorn\Anaconda3\lib\site-package

GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'class_weight': ['dict', 'balanced'],
                         'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [0.001, 0.01, 0.1, 1.0, 10.0,
                                              100.0, 1000.0],
                         'min_samples_split': [2, 5, 1.0, 20, 50, 100],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted')

In [11]:
# print best parameter after tuning 
#print(grid.best_params_) 


In [12]:
re=grid.cv_results_
print(re)


{'mean_fit_time': array([1.15876198e-03, 8.66460800e-04, 7.02285767e-04, ...,
       1.38306618e-03, 1.61604881e-03, 3.57627869e-05]), 'std_fit_time': array([1.09390388e-03, 1.73292160e-03, 1.40457153e-03, ...,
       1.71767266e-03, 3.23209763e-03, 7.15255737e-05]), 'mean_score_time': array([0., 0., 0., ..., 0., 0., 0.]), 'std_score_time': array([0., 0., 0., ..., 0., 0., 0.]), 'param_class_weight': masked_array(data=['dict', 'dict', 'dict', ..., 'balanced', 'balanced',
                   'balanced'],
             mask=[False, False, False, ..., False, False, False],
       fill_value='?',
            dtype=object), 'param_criterion': masked_array(data=['gini', 'gini', 'gini', ..., 'log_loss', 'log_loss',
                   'log_loss'],
             mask=[False, False, False, ..., False, False, False],
       fill_value='?',
            dtype=object), 'param_max_features': masked_array(data=['auto', 'auto', 'auto', ..., 'log2', 'log2', 'log2'],
             mask=[False, False, False, .

In [13]:
grid_predictions = grid.predict(x_test) 

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
cm


array([[50,  1],
       [ 2, 80]], dtype=int64)

In [14]:

# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)
print(clf_report)

              precision    recall  f1-score   support

           0       0.96      0.98      0.97        51
           1       0.99      0.98      0.98        82

    accuracy                           0.98       133
   macro avg       0.97      0.98      0.98       133
weighted avg       0.98      0.98      0.98       133



In [15]:

from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'class_weight': 'balanced', 'criterion': 'gini', 'max_features': 'log2', 'min_samples_leaf': 0.001, 'min_samples_split': 5, 'splitter': 'random'}: 0.9774839146827697


In [16]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])


0.978000956480153

In [17]:
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)
print((cm))
print(clf_report)

The f1_macro value for best parameter {'class_weight': 'balanced', 'criterion': 'gini', 'max_features': 'log2', 'min_samples_leaf': 0.001, 'min_samples_split': 5, 'splitter': 'random'}: 0.9774839146827697
[[50  1]
 [ 2 80]]
              precision    recall  f1-score   support

           0       0.96      0.98      0.97        51
           1       0.99      0.98      0.98        82

    accuracy                           0.98       133
   macro avg       0.97      0.98      0.98       133
weighted avg       0.98      0.98      0.98       133



In [18]:
table=pd.DataFrame.from_dict(re)

In [20]:
table.shape

(1512, 19)

In [21]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_criterion,param_max_features,param_min_samples_leaf,param_min_samples_split,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001159,0.001094,0.0,0.0,dict,gini,auto,0.001,2,best,"{'class_weight': 'dict', 'criterion': 'gini', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,409
1,0.000866,0.001733,0.0,0.0,dict,gini,auto,0.001,2,random,"{'class_weight': 'dict', 'criterion': 'gini', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1012
2,0.000702,0.001405,0.0,0.0,dict,gini,auto,0.001,5,best,"{'class_weight': 'dict', 'criterion': 'gini', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1011
3,0.001561,0.002086,0.0,0.0,dict,gini,auto,0.001,5,random,"{'class_weight': 'dict', 'criterion': 'gini', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1010
4,0.000829,0.001657,0.0,0.0,dict,gini,auto,0.001,1,best,"{'class_weight': 'dict', 'criterion': 'gini', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1507,0.002689,0.002393,0.0,0.0,balanced,log_loss,log2,1000,20,random,"{'class_weight': 'balanced', 'criterion': 'log...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,520
1508,0.003320,0.002245,0.0,0.0,balanced,log_loss,log2,1000,50,best,"{'class_weight': 'balanced', 'criterion': 'log...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,519
1509,0.001383,0.001718,0.0,0.0,balanced,log_loss,log2,1000,50,random,"{'class_weight': 'balanced', 'criterion': 'log...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,518
1510,0.001616,0.003232,0.0,0.0,balanced,log_loss,log2,1000,100,best,"{'class_weight': 'balanced', 'criterion': 'log...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1417
